In [1]:
import os
# change the default cache dir so that huggingface won't take the cse space.
os.environ['TRANSFORMERS_CACHE'] = '/export/scratch/zeren/KimNLP/HuggingfaceCache/'

from utils import save_args, select_activation_fn
from old_train import Trainer
from old_data import create_data_channels
from old_model import BertClassifier
from torch.utils.data import DataLoader
from transformers import AutoModel
import numpy as np
import random
import torch
import argparse

/export/scratch/Zeren/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
def collate_fn(samples):
    samples = list(map(list, zip(*samples)))
    return samples

In [35]:
data_dir = '../../NewData/'
data_filename = os.path.join(data_dir, 'old_kim.tsv')
model_name = '../../Pre-Trained-SciBERT/checkpoint-213120/'

token_train_data, token_val_data, token_test_data = create_data_channels(
    data_filename,
    model_name,
    max_length=512
)

Number of data instance: 805
Tokenizing citation contexts.


100%|██████████| 563/563 [00:00<00:00, 1081.74it/s]


Tokenizing citing abstracts.


100%|██████████| 563/563 [00:00<00:00, 1459.26it/s]


Tokenizing cited abstracts.


...

100%|██████████| 80/80 [00:00<00:00, 810.51it/s]


Tokenizing citing abstracts.


100%|██████████| 80/80 [00:00<00:00, 1558.71it/s]


Tokenizing cited abstracts.


100%|██████████| 80/80 [00:00<00:00, 1446.52it/s]


In [36]:
test_loader = DataLoader(token_test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)
dliterator = iter(test_loader)

In [5]:
model_filename = 'Workspaces/test_old_pre_trained_scibert/best_model.pt'
model = torch.load(model_filename).cpu()

In [38]:
citation_context, citing_abstract, cited_abstract, label = next(dliterator)
print(citation_context[0], citing_abstract[0], cited_abstract[0], label)

[{'input_ids': tensor([[  102,   131,   629,   579,  3781,  9942, 30113,   137,  1337,   111,
          2923,   467,   111,   158,  1352,   231,  4899,   131,  4211,  8499,
           131,   629,   579,  3781,  9942, 30113,   137,   547,  8391,  2168,
          7904,   579,  5677,  3212,   205,   121,   238,  1203,   422,   185,
           709,   580,  4142, 15688,  3853,   168, 25462,   422,  3526,  1445,
          1017,   111,  2220,   131,   238,  2151,   697,   214,  7402,   903,
           365,   186,   205,   260,  1107,  1901,   188,   804,   188,  3978,
           111,  7246,   131,   238,   527,  1017,   111,  1073,  2501,   191,
         25462,   137, 10937,  9320,  3874, 10000,   205,   111,  6226,  8499,
           267,  7180,   214,  2271,  3277,  3853,   487,  1135,   579,  2211,
          7479,  1352,  2271, 10133,   179,  3098, 11128,   145,  5572,  5803,
         30113,   546,   205,   260,  1557,  1901,   355,   131,   111,  3853,
           267,  1260,   191,  4039, 

In [13]:
model.eval()

BertClassifier(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=3840, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=8, bias=True)
  )
  (batch_norm): ModuleList(
    (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (top_layer): Linear(in_features=8, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (scibert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encod

In [39]:
with torch.no_grad():
    for i in range(len(citation_context[0])):
        citation_context[0][i]['readout_mask'] = None
    preds = model(
        None,
        cited_abstract[0],
        None,
        citing_abstract[0],
        citation_context[0]
    )
print(preds)

tensor([[-0.7502,  1.0077]])


In [40]:
labels = []
preds = []
with torch.no_grad():
    for citation_context, citing_abstract, cited_abstract, label in test_loader:
        for i in range(len(citation_context[0])):
            citation_context[0][i]['readout_mask'] = None
        preds.append(model(
            None,
            cited_abstract[0],
            None,
            citing_abstract[0],
            citation_context[0]
        ).detach().cpu().numpy())
        labels.append(label)

preds = np.concatenate(preds, axis=0)
labels = np.concatenate(labels, axis=0)

In [41]:
preds, labels

(array([[ 0.30269256, -0.63129145],
        [-0.75021625,  1.0077335 ],
        [-0.2956124 ,  0.2644648 ],
        [ 0.26255557, -0.4977579 ],
        [ 0.3025941 , -0.667239  ],
        [-0.5657457 ,  0.7059342 ],
        [ 0.2744343 , -0.61456114],
        [ 0.21550709, -0.51385987],
        [ 0.3190036 , -0.660746  ],
        [-0.72832274,  1.0440961 ],
        [-0.73572373,  1.0859519 ],
        [ 0.30726567, -0.6460807 ],
        [ 0.31361094, -0.665917  ],
        [-0.7007373 ,  1.0490079 ],
        [ 0.30634257, -0.6822105 ],
        [ 0.31052265, -0.67988986],
        [ 0.30557153, -0.6421743 ],
        [-0.71239424,  1.0009856 ],
        [-0.15290911,  0.10688871],
        [ 0.31224427, -0.67054915],
        [-0.6898694 ,  1.0227102 ],
        [ 0.29373345, -0.69220215],
        [ 0.2731047 , -0.63055223],
        [-0.7146323 ,  1.0783947 ],
        [-0.6932931 ,  1.0333409 ],
        [-0.6335604 ,  0.91107327],
        [ 0.28706482, -0.66667753],
        [-0.20882446, -0.021

In [32]:
from sklearn.metrics import f1_score

In [42]:
f1_score(labels, preds.argmax(axis=1), average='macro')

0.7527472527472527